In [9]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [16]:
train_path = 'data/signup_train_data.csv'
test_path = 'data/signup_test_data.csv'


# Load the data into a DataFrame 
df = pd.read_csv(train_path, low_memory=False)
test_df = pd.read_csv(test_path, low_memory=False)

# Split into X and Y
dfLabels = df.pop("label")
#print (df.head())

#Feature Engineering
#for index, row in df.iterrows():
#    df['time_of_day_int'] = int(row['created_at'][11:13])*3600 + int(row['created_at'][14:16])*60 + int(row['created_at'][17:19])

df['employee_count_low'] = int(df['employee_count'].str.split()[0][0])
df['employee_count_high'] = int(df['employee_count'].str.split()[0][2])
test_df['employee_count_low'] = int(df['employee_count'].str.split()[0][0])
test_df['employee_count_high'] = int(df['employee_count'].str.split()[0][2])

#Temporarily parse non numerical data
frames_to_remove = ['zip', 'city', 'website', 'state', 'country', 'registration_ip', 'company','multifactor_country_code','created_at','lead_source','marketing_channel','volume','user_persona','initial_package','employee_count','geolocation_notes','name_notes','ip_notes','community_notes','email_notes','activity_notes','fingerprint_notes']
df = df.drop(frames_to_remove, axis=1)
test_df = test_df.drop(frames_to_remove, axis=1)

df.fillna(value=0, inplace=True)

df.head()

,id,banned_ip,ip_count,is_authy_verified,banned_email,is_transactional,is_marketing,is_behavioral,is_oem,risk,...,mfa_required,mfa_completed,whitelabel_required,whitelabel_completed,payment_required,payment_completed,profile_completed,email_completed,employee_count_low,employee_count_high
0,0,6,8,0,1,0,0,0,0,-100,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1,500
1,1,5,5,0,1,1,0,0,0,-90,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,500
2,2,0,1,0,0,1,0,0,0,-44,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,500
3,3,4,4,0,2,0,1,0,0,-72,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1,500
4,4,0,1,0,0,1,0,0,0,-34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,500


In [19]:
def GnbBasic(seed_in, TO_TEST, split=0.5):
    # split data into train and test sets
    seed = seed_in
    X_train, X_test, y_train, y_test = train_test_split(df, dfLabels, test_size=1-split, random_state=seed)

    # fit model no training data
    model = GaussianNB()
    model.fit(X_train, y_train)
    
    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]

    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("Seed: %i - Accuracy: %.3f%%" % (seed, accuracy * 100.0))
    
    if TO_TEST:
        #make predictions for train data
        y_pred = model.predict(test_df)
        predictions = [round(value) for value in y_pred]

        d = {'id':test_df['id'], 'label':predictions}
        preds = pd.DataFrame(data=d)

        preds.to_csv('predictions.csv', index=False)

In [20]:
GnbBasic(1, False)

Seed: 1 - Accuracy: 72.886%
